## Vector Stores and Retrievers

In [32]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [33]:
groqApiKey = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [34]:
from langchain_groq     import ChatGroq

LLM = ChatGroq(groq_api_key = groqApiKey, model = "Llama3-8b-8192")
LLM

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x3325b6080>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x3325b50c0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [35]:
from langchain_huggingface      import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

In [36]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [37]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [38]:
from langchain_chroma       import Chroma

vectorStore = Chroma.from_documents(documents, embedding = embeddings)
vectorStore

In [39]:
vectorStore.similarity_search("cat")

[Document(id='24a78b84-83da-4417-88e4-8d15a39b5f85', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='49172060-3c13-4902-a921-53b42df6f87e', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='74c07fb3-1ba8-42bf-95cd-f4e0ace2874a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='edc60747-d186-4f8a-bc40-6c2a207af4cb', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [40]:
vectorStore.similarity_search_with_score("cat")

[(Document(id='24a78b84-83da-4417-88e4-8d15a39b5f85', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351050853729248),
 (Document(id='49172060-3c13-4902-a921-53b42df6f87e', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351050853729248),
 (Document(id='74c07fb3-1ba8-42bf-95cd-f4e0ace2874a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351050853729248),
 (Document(id='edc60747-d186-4f8a-bc40-6c2a207af4cb', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740904808044434)]

## Async Query

In [41]:
await vectorStore.asimilarity_search("cat")

[Document(id='24a78b84-83da-4417-88e4-8d15a39b5f85', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='49172060-3c13-4902-a921-53b42df6f87e', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='74c07fb3-1ba8-42bf-95cd-f4e0ace2874a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='edc60747-d186-4f8a-bc40-6c2a207af4cb', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

## Retriever

In [44]:
from typing import List
from langchain_core.documents       import Document
from langchain_core.runnables       import RunnableLambda

retriever = RunnableLambda(vectorStore.similarity_search).bind(k = 1)
retriever.batch(["cat", "dog"])

[[Document(id='74c07fb3-1ba8-42bf-95cd-f4e0ace2874a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='aef8d61a-3e61-42fa-8acc-95efb86b3793', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [45]:
retriever  = vectorStore.as_retriever(
    search_type     = "similarity",
    search_kwargs   = {"k": 1}
)

retriever.batch(["cat", "dog"])

[[Document(id='74c07fb3-1ba8-42bf-95cd-f4e0ace2874a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='aef8d61a-3e61-42fa-8acc-95efb86b3793', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

## RAG

In [46]:
from langchain_core.prompts         import ChatPromptTemplate
from langchain_core.runnables       import RunnablePassthrough

message = """
Answer this question using the provided text only.
{question}

Content:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

In [47]:
ragChain = {"context": retriever, "question": RunnablePassthrough()}|prompt|LLM
response = ragChain.invoke("tell me something about dogs")

In [48]:
response.content

'According to the provided text, dogs are known for their loyalty and friendliness.'